In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency 

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import reverb
import scipy.stats as stats
import tensorflow as tf
import tensorflow_probability as tfp
import tf_agents.environments as tfae

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import tensor_spec
from tf_agents.specs import array_spec
from tf_agents.utils import common

tfd = tfp.distributions

In [3]:
num_iterations = 10000 # @param {type:"integer"}

initial_collect_steps = 10  # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_max_length = 10000  # @param {type:"integer"}

batch_size = 32  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 100  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 500  # @param {type:"integer"}

In [4]:
T = 20
NUM_STEPS = 20 * 1
dt = T / NUM_STEPS
MU = 0.05 / 250
IR = MU
SIGMA = 0.20 / np.sqrt(250)
S0 = 100
K = 100
NUM_SOLD_OPT = 100
KAPPA = 0.1
GAMMA = 0.9
c = 0.001

In [5]:
class GBM:
    
      def StockPrice(dt, n_paths, n_steps, mu, sigma, S0, seed=0):

        S = np.zeros((n_steps,n_paths))
        W = np.zeros((n_steps,n_paths))
        S[0] = S0
        for t in range(1,n_steps):
            dW = np.sqrt(dt) * np.random.standard_normal(n_paths)
            W[t] = W[t-1] + dW
            S[t] = S[t-1] * np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * dW)

        return S.T, W.T

      def brownian_sim(num_path, num_period, mu, std, init_p, dt):
        z = np.random.normal(size=(num_path, num_period))

        a_price = np.zeros((num_path, num_period))
        a_price[:, 0] = init_p

        for t in range(num_period - 1):
            a_price[:, t + 1] = a_price[:, t] * np.exp(
                (mu - (std ** 2) / 2) * dt + std * np.sqrt(dt) * z[:, t]
            )
        return a_price

In [6]:
class BSM:
  
    def OptionPrice(S, K, ir, sigma, T):
    
        prices = np.zeros_like(S)
        deltas = np.zeros_like(S)
        for t in range(S.shape[1]): 
            ttm = T - t
  
      # Before expiry
        if ttm > 0:
            d1 = (np.log(S[:, t]/K) + (ir + sigma**2 / 2) * ttm) / (sigma * np.sqrt(ttm))
            d2 = (np.log(S[:, t]/K) + (ir - sigma**2 / 2) * ttm) / (sigma * np.sqrt(ttm))
            prices[:, t] = S[:, t] * stats.norm.cdf(d1) - K * np.exp(-ir * ttm) * stats.norm.cdf(d2)
            deltas[:, t] = stats.norm.cdf(d1)

      # At expiry
        else:
            prices[:, -1] = np.where(S[:, t] > K, S[:, t] - K, 0.0)
            deltas[:, -1] = np.where(S[:, t] > K, 1.0, 0.0)
    
        return prices, deltas

In [7]:
class EnvironmentBS(tfae.py_environment.PyEnvironment):
  
    def __init__(self, T, n_steps, mu, ir, sigma, S0, K, num_sold_opt, kappa, c):
        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=num_sold_opt, name="action")
        self._observation_spec = array_spec.ArraySpec(shape=(5,), dtype=np.float32, name="observation")
        self.T = T
        self.n_steps = n_steps
        self.mu = mu
        self.ir = ir
        self.sigma = sigma
        self.S0 = S0
        self.K = K
        self.num_sold_opt = num_sold_opt
        self.kappa = kappa
        self.seed = 0
        self.c = c

        self.dt = T / n_steps
        self.initialize_paths = False
        self._episode_ended = False

    def action_spec(self):
      self._action_space = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, 
                  minimum=0, maximum=100, name="action")
      return self._action_space

    def observation_spec(self):
      self._observation_spec = array_spec.ArraySpec(shape=(5,), dtype=np.float32, name="observation")
      return self._observation_spec

    def __generate_paths(self, n_paths=10000):
      
        self.n_paths = n_paths
        self.idx_path = -1

        # Make stock paths
        S_paths, W_paths = GBM.StockPrice(self.dt, self.n_paths, self.n_steps+1, self.mu, self.sigma, self.S0, self.seed)

        # Make time paths
        t_paths = np.zeros_like(S_paths)
        for j in range(S_paths.shape[1]): t_paths[:, j] = self.dt * j

        # Make option prices and deltas paths
        call_opt_paths, delta_paths = BSM.OptionPrice(S_paths, self.K, self.ir, self.sigma, self.T)

        self.S_paths = S_paths
        self.W_paths = W_paths
        self.t_paths = t_paths
        self.call_opt_paths = call_opt_paths
        self.delta_paths = delta_paths

    def __get_state_without_num_stocks(self, i_path, j_time):

        t = self.t_paths[i_path, j_time]
        W = self.W_paths[i_path, j_time]
        C = self.call_opt_paths[i_path, j_time]
        delta = self.delta_paths[i_path, j_time] * self.num_sold_opt # multiplied by num options
        num_stk = 0

        return np.array([t, W, C, delta, num_stk],dtype="float32") # state


    def clear_all_paths(self):
        self.initialize_paths = False

    def _reset(self):

        if not self.initialize_paths:
            self.__generate_paths()
            self.initialize_paths = True

        self.idx_path = (self.idx_path + 1) % self.n_paths
        self.idx_time = 0
        self._episode_ended = False

        state = self.__get_state_without_num_stocks(self.idx_path, self.idx_time)
        self.state = state

        return ts.restart(self.state)

    def _step(self, action):
        if self._episode_ended:
          # The last action ended the episode. Ignore the current action and start
          # a new episode.
            return self.reset()

        if self.idx_time > self.n_steps:
            n_state = None
            r = np.nan
            done = True
            self._episode_ended = True
            info = None

        elif self.idx_time == self.n_steps:
            n_state = self.__get_state_without_num_stocks(self.idx_path, self.idx_time)
            r = self.__get_reward(n_state)
            done = True
            self._episode_ended = True

        else:
            self.idx_time += 1
            n_state = self.__get_state_without_num_stocks(self.idx_path, self.idx_time)
            n_state[4] = action # num of stocks is updated.
            r = self.__get_reward(n_state)
            done = False

        self.state = n_state

        if self._episode_ended:
          return ts.termination(self.state, reward=r)
        else:
          return ts.transition(self.state, reward=r)

    def __get_reward(self, n_state=None):

        

        t1 = n_state[0]
        t0 = self.state[0]

        S1 =  self.S0 * np.exp(self.sigma * n_state[1] + (self.mu - self.sigma**2 / 2) * t1)
        S0 = self.S0 * np.exp(self.sigma * self.state[1] + (self.mu - self.sigma**2 / 2) * t0)

        C1 = n_state[2]
        C0 = self.state[2]

        d1 = n_state[3] # = delta per one option * num sold options
        d0 = self.state[3] # = delta per one option * num sold options

        nS1 = n_state[4]
        nS0 = self.state[4]

        
        r = (nS1 * S1 - nS0 * S0)
        r += self.num_sold_opt * (C1 - C0)

        
        #r -= (nS1 - nS0) * S0 * np.exp(self.ir * (self.T - t0))


        if self.kappa > 0:
          r -= self.kappa * r**2
        
        if self.c > 0:
          r -= self.c * np.abs(S1*(nS1 - nS0))

        if n_state is None or self.state[0] == self.T:
          r = (nS1 * S1 - nS0 * S0) - (max(S1 - self.K, 0) - C1) * self.num_sold_opt * 100 - nS1 * S1 * self.c
          r = r - self.kappa * r**2
          r = r - self.c * np.abs(S1*(nS1 - nS0))
          

        return r

In [8]:
env = EnvironmentBS(T, NUM_STEPS, MU, IR, SIGMA, S0, K, NUM_SOLD_OPT, KAPPA, c)
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Reward Spec:')
print(env.time_step_spec().reward)
print('Action Spec:')
print(env.action_spec())

Observation Spec:
ArraySpec(shape=(5,), dtype=dtype('float32'), name='observation')
Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')
Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=100)


In [9]:
train_py_env = env
eval_py_env = env
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [10]:
train_env.reset()

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[0., 0., 0., 0., 0.]], dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>})

In [11]:
fc_layer_params = (16, 16)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1.
num_actions

101.0

In [12]:
# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
    return tf.keras.layers.Dense(num_units,
    activation=tf.keras.activations.relu,
    kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [14]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [15]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),train_env.action_spec())

In [16]:
train_env.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(100, dtype=int32))

In [17]:
# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
    return tf.keras.layers.Dense(
        num_units,
        activation=tf.keras.activations.relu,
        kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [18]:
def compute_avg_return(environment, policy, num_episodes=10):

    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

compute_avg_return(eval_env, random_policy, num_episodes=10)

-42663336.0

In [19]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

In [20]:
agent.collect_data_spec

Trajectory(
{'action': BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(100, dtype=int32)),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': TensorSpec(shape=(5,), dtype=tf.float32, name='observation'),
 'policy_info': (),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [21]:
agent.collect_data_spec._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

In [22]:
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_py_env.reset())

(TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([10.       , -0.6445507,  0.       ,  0.       , 44.       ],
      dtype=float32),
 'reward': array(-9012.84, dtype=float32),
 'step_type': array(1, dtype=int32)}),
 ())

In [23]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

<PrefetchDataset element_spec=(Trajectory(
{'action': TensorSpec(shape=(32, 2), dtype=tf.int32, name=None),
 'discount': TensorSpec(shape=(32, 2), dtype=tf.float32, name=None),
 'next_step_type': TensorSpec(shape=(32, 2), dtype=tf.int32, name=None),
 'observation': TensorSpec(shape=(32, 2, 5), dtype=tf.float32, name=None),
 'policy_info': (),
 'reward': TensorSpec(shape=(32, 2), dtype=tf.float32, name=None),
 'step_type': TensorSpec(shape=(32, 2), dtype=tf.int32, name=None)}), SampleInfo(key=TensorSpec(shape=(32, 2), dtype=tf.uint64, name=None), probability=TensorSpec(shape=(32, 2), dtype=tf.float64, name=None), table_size=TensorSpec(shape=(32, 2), dtype=tf.int64, name=None), priority=TensorSpec(shape=(32, 2), dtype=tf.float64, name=None), times_sampled=TensorSpec(shape=(32, 2), dtype=tf.int32, name=None)))>

In [24]:
iterator = iter(dataset)
print(iterator)

In [25]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, 1)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
step = 100: loss = 4418828763136.0
step = 200: loss = 1842630492160.0
step = 300: loss = 2842637369344.0
step = 400: loss = 4082755174400.0
step = 500: loss = 8473125847040.0
step = 500: Average Return = -3717875.25
step = 600: loss = 2328952832000.0
step = 700: loss = 6617654362112.0
step = 800: loss = 7816913879040.0
step = 900: loss = 6074636173312.0
step = 1000: loss = 3420306538496.0
step = 1000: Average Return = -13598413.0
step = 1100: loss = 390752567296.0
step = 1200: loss = 3230326849536.0
step = 1300: loss = 7664071344128.0
step = 1400: loss = 3574668984320.0
step = 1500: loss = 4012868108288.0
step = 1500: Average Return = -18583332.0
step = 1600: loss = 1400268914688.0
step = 1700